https://github.com/karndeepsingh/Named-Entity-Recognition/blob/main/NAMED%20ENTITY%20RECOGNITION.ipynb

# 01_ Load modules

In [1]:
# !pip install simpletransformers

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel,NERArgs
import pandas as pd 
import numpy as np

/home/e077926/miniconda3/Bib/envs/tata_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-24 15:27:46.657629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 15:27:46.657671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 15:27:46.658402: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 15:27:46.662808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Te

In [4]:
df=pd.read_csv('/home/e077926/buscode_2023/05_E2_tata/data/bio_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,doc,line_id,value,entity
0,X00016469612,1,tan,O
1,X00016469612,1,woon,O
2,X00016469612,1,yann,O
3,X00016469612,2,book,company-B
4,X00016469612,2,tak,O


# 02_ encode doc and lines

In [5]:
df_doc_encod=df.copy()
# Créer une nouvelle colonne 'sentence' en combinant 'doc' et 'line_id'
df_doc_encod['sentence_id'] = df_doc_encod['doc'] + df_doc_encod['line_id'].astype(str)
df_doc_encod['words']=df_doc_encod['value']
df_doc_encod['labels']=df_doc_encod['entity']
df_doc_encod=df_doc_encod.drop(columns=['doc','line_id', 'value', 'entity'])
df_doc_encod["sentence_id"] = LabelEncoder().fit_transform(df_doc_encod["sentence_id"] )

df_doc_encod.tail(10)

,sentence_id,words,labels
66821,17992,cashier,O
66822,17992,cashier,O
66823,18056,cashier,O
66824,18056,cashier,O
66825,18086,cashier,O
66826,18086,cashier,O
66827,18114,cashier,O
66828,18114,cashier,O
66829,18138,rm,O
66830,18138,rm,O


In [6]:
df_doc_encod["sentence_id"] = LabelEncoder().fit_transform(df_doc_encod["sentence_id"] )

df_doc_encod.tail(10)

,sentence_id,words,labels
66821,17992,cashier,O
66822,17992,cashier,O
66823,18056,cashier,O
66824,18056,cashier,O
66825,18086,cashier,O
66826,18086,cashier,O
66827,18114,cashier,O
66828,18114,cashier,O
66829,18138,rm,O
66830,18138,rm,O


In [7]:
df_doc_encod['labels']=df_doc_encod['labels'].map({'O':'O',
                          'address-B': 'B-ADD',
                          'address-I': 'I-ADD',
                          'date-B': 'B-TIM','date-I': 'I-TIM',
                          'company-B': 'B-ORG',
                          'company-I': 'I-ORG',
                          'total-B':'B-TOT','total-I':'I-TOT'})

In [7]:
# df_doc_encod['sentence_id']=df_doc_encod['doc']+df_doc_encod['line_id']
# df_doc_encod=df_doc_encod.drop(columns=['doc', 'line_id'])
# df_doc_encod=df_doc_encod.sort_values(by=['sentence_id', 'value', 'entity'])
# df_doc_encod.tail(10)

In [8]:
ner_tags_tuple_list = [('O', 0),
            ('B-ORG', 1),
            ('I-ORG', 2),
            ('B-ADD', 3),
            ('I-ADD', 4),
            ('B-TIM', 5),
            ('I-TOT', 6),
            ('B-TOT', 7),
            ('I-TIM', 8)]

# Création du dictionnaire
tag2index = {key: value for key, value in ner_tags_tuple_list}
index2tag = {value: key for key, value in ner_tags_tuple_list}
index2tag

{0: 'O',
 1: 'B-ORG',
 2: 'I-ORG',
 3: 'B-ADD',
 4: 'I-ADD',
 5: 'B-TIM',
 6: 'I-TOT',
 7: 'B-TOT',
 8: 'I-TIM'}

In [9]:
df_doc_encod['ner_tags']=df_doc_encod['labels'].map(tag2index)
df_doc_encod.head()

,sentence_id,words,labels,ner_tags
0,0,tan,O,0
1,0,woon,O,0
2,0,yann,O,0
3,11,book,B-ORG,1
4,11,tak,O,0


# 03_ Split data

In [8]:
X=df_doc_encod[["sentence_id", "words"]]
Y=df_doc_encod["ner_tags"]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [10]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"ner_tags":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"ner_tags":y_test})

# 04 Model Training

In [16]:
labels= df_doc_encod["ner_tags"].unique().tolist()
labels

[0, 1, 2, 3, 4, 5, 7, 6, 8]

In [13]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [17]:
model = NERModel('bert', 'bert-base-cased',labels=labels,args =args)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

KeyError: 'labels'

# Test

https://github.com/laxmimerit/NLP-Tutorials-with-HuggingFace/blob/main/2%20NER%20Training%20%7C%20NLP%20with%20HuggingFace%20Tutorial.ipynb

In [13]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
ner_tags_tuple_list = [('O', 0),
            ('B-ORG', 1),
            ('I-ORG', 2),
            ('B-ADD', 3),
            ('I-ADD', 4),
            ('B-TIM', 5),
            ('I-TOT', 6),
            ('B-TOT', 7),
            ('I-TIM', 8)]

# Création du dictionnaire
tag2index = {key: value for key, value in ner_tags_tuple_list}
index2tag = {value: key for key, value in ner_tags_tuple_list}
index2tag

{0: 'O',
 1: 'B-ORG',
 2: 'I-ORG',
 3: 'B-ADD',
 4: 'I-ADD',
 5: 'B-TIM',
 6: 'I-TOT',
 7: 'B-TOT',
 8: 'I-TIM'}

In [10]:
df_doc_encod['ner_tags']=df_doc_encod['labels'].map(tag2index)
df_doc_encod.head()

,sentence_id,words,labels,ner_tags
0,0,tan,O,0
1,0,woon,O,0
2,0,yann,O,0
3,11,book,B-ORG,1
4,11,tak,O,0


In [11]:
df_doc_encod.to_csv('/home/e077926/buscode_2023/05_E2_tata/data/final_dataset.csv')

In [19]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
                                                    model_checkpoint,
                                                    id2label=index2tag,
                                                    label2id=tag2index)

model.safetensors: 100%|██████████| 263M/263M [01:00<00:00, 4.32MB/s] 
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments

args = TrainingArguments("distilbert-finetuned-ner",
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=3,
                         weight_decay=0.01)

## split

In [22]:
X=df_doc_encod[["sentence_id", "words"]]
Y=df_doc_encod["ner_tags"]
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

## Metrics

In [23]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
# !pip install seqeval
# !pip install evaluate

import evaluate
metric = evaluate.load('seqeval')

In [28]:
label_names=df_doc_encod['labels'].unique().tolist()

In [29]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

In [30]:
from transformers import Trainer
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = train_data,
                  eval_dataset = test_data,
                #   data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

# 05_ random forest classifier

In [ ]:
def feature_map(word):
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])

In [ ]:
words = [feature_map(w) for w in df_doc_encod["words"].values.tolist()]
tags = df_doc_encod["labels"].values.tolist()

AttributeError: 'float' object has no attribute 'istitle'

In [ ]:
def feature_map(word):
    if isinstance(word, str):
        return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                         word.isdigit(), word.isalpha()])
    else:
        # Handle the case when the word is not a string (e.g., a float)
        return np.zeros(6)  # Return a default array of zeros for non-string values

words = [feature_map(w) for w in df_doc_encod["words"].values.tolist()]
tags = df_doc_encod["labels"].values.tolist()


In [ ]:
words[:5]

[array([0, 1, 0, 3, 0, 1]),
 array([0, 1, 0, 4, 0, 1]),
 array([0, 1, 0, 4, 0, 1]),
 array([0, 1, 0, 4, 0, 1]),
 array([0, 1, 0, 3, 0, 1])]

In [ ]:
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),X=words, y=tags, cv=5)

In [ ]:
#Lets check the performance 
from sklearn.metrics import classification_report
report = classification_report(y_pred=pred, y_true=tags)
print(report)

/home/e077926/miniconda3/Bib/envs/tata_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/e077926/miniconda3/Bib/envs/tata_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-ADD       0.00      0.00      0.00      1491
       B-ORG       0.00      0.00      0.00       875
       B-TIM       0.57      0.73      0.64       488
       B-TOT       0.00      0.00      0.00       975
       I-ADD       0.46      0.02      0.04      4188
       I-ORG       0.00      0.00      0.00      1470
       I-TIM       0.00      0.00      0.00       146
       I-TOT       0.00      0.00      0.00        23
           O       0.86      0.99      0.92     57175

    accuracy                           0.86     66831
   macro avg       0.21      0.19      0.18     66831
weighted avg       0.77      0.86      0.80     66831



/home/e077926/miniconda3/Bib/envs/tata_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# https://www.kaggle.com/code/bavalpreet26/ner-using-crf